## Spatial interpolation and approximation I: methods
Resources:

* [
GRASS GIS overview and manual](http://grass.osgeo.org/grass72/manuals/index.html)
*  [Recommendations](data_acquisition.html#commands)
and [tutorial](http://www4.ncsu.edu/~akratoc/GRASS_intro/)
how to use wxGUI from the first assignment


### Start GRASS GIS
Start GRASS - click on GRASS icon or type

In [ ]:
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"]).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/home/jovyan/grassdata", "nc_spm_08_grass7", "user1")

In [ ]:
# a proper directory is already set, download files
import urllib

In [ ]:
# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

In [ ]:
# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'

In startup pannel set GIS Data Directory to path to datasets,
for example on MS Windows, `C:\Users\myname\grassdata`.
For Project location select nc_spm_08_grass7 (North Carolina, State Plane, meters) and
for Accessible mapset create a new mapset (called e.g. HW_interpolation_1).
Click Start GRASS.


### Compute Voronoi polygons
Display the polygons with centroids.
Find the column name where z is stored and convert the polygons to raster.
Compute aspect to evaluate the surface geometry.

In [ ]:
gs.parse_command('g.region', region="rural_1m", flags='pg')
gs.run_command('d.erase')
gs.run_command('v.voronoi', input="elev_lid792_randpts", output="elev_vor")
gs.run_command('d.vect', map="elev_vor", size="1", type="area,centroid", flags='c')
gs.read_command('v.info', map="elev_vor", flags='c')
gs.run_command('v.to.rast', input="elev_vor", output="elev_vor_1m", attrcolumn="value", use="attr")
gs.run_command('r.colors', map="elev_vor_1m", color="elevation")
gs.run_command('r.slope.aspect', elevation="elev_vor_1m", aspect="asp_vor_1m")
Image(filename="map.png")

Display the resulting elevation map and aspect map.

In [ ]:
gs.run_command('d.erase')
gs.run_command('d.rast', map="elev_vor_1m")
gs.run_command('d.rast', map="asp_vor_1m")
gs.run_command('d.vect', map="elev_lid792_randpts", size="1", color="red")
Image(filename="map.png")

Optionally, you can view it in 3D perspective
(switch off all layers except for elev_vor_1m and switch to 3D view).

### Interpolation using IDW
Set region and resolution, find a column name where z is stored.
Interpolate DEM using IDW, check the result using aspect.

In [ ]:
gs.parse_command('g.region', region="rural_1m", flags='pg')
gs.read_command('v.info', map="elev_lid792_randpts", flags='c')
gs.run_command('v.surf.idw', input="elev_lid792_randpts", output="elev_idw_1m", column="value")
gs.run_command('r.colors', map="elev_idw_1m", color="elevation")
gs.run_command('r.slope.aspect', elevation="elev_idw_1m", aspect="asp_idw_1m")
gs.run_command('d.erase')
gs.run_command('d.rast', map="elev_idw_1m")
gs.run_command('d.rast', map="asp_idw_1m")
gs.run_command('d.vect', map="elev_lid792_randpts", size="2", color="red")
Image(filename="map.png")

Design experiment that elucidates the impact of IDW parameters on the surface,
focus on the impact of:

* exponent e.g., power=0.5, 1, 5 (2 is the default)
* number of neighboring points e.g., npoint=1, 5, 20, 60 (12 is the default)

Include selected images (e.g. hillshade or aspect)
and _relevant_ stats (e.g., mean, min, max from _r.univar_, histogram)
that highlight the differences in the resulting surfaces into your report.

Check the surface interpolated with default parameters using 3D view.
Do not forget to switch off everything except for the interpolated elevations and
set fine resolution to 1.
You can use constant color for the surface to highlight its structure.
Save an image for your report.

### Compute DEM from contours
Compute DEM from contours using linear interpolation between isolines:

In [ ]:
gs.parse_command('g.region', region="rural_1m", flags='pg')
gs.run_command('v.to.rast', input="elev_lid792_cont1m", output="el_lid792_cont1m", attrcolumn="level", use="attr")
gs.run_command('r.surf.contour', input="el_lid792_cont1m", output="el_rcont")
gs.run_command('r.colors', map="el_rcont", color="elevation")

Check the result using a 2D aspect map or view el_rcont in 3D.
In 3D set view from SE and light from NW to reveal subtle geometry.

In [ ]:
gs.run_command('r.slope.aspect', elevation="el_rcont", aspect="asp_rcont")
gs.run_command('d.rast', map="el_rcont")
gs.run_command('d.rast', map="asp_rcont")
gs.run_command('d.vect', map="elev_lid792_cont1m", col="white")
Image(filename="map.png")

### Optional: create TIN model
Convert z-value stored as attribute "value" to z-coordinate.
Compute TIN and visualize it as 3D vector data:
Keep only "level90", "elev_idw_1m" and "elev_rand_tin" switched on.
For "elev_rand_tin" > _Properties_ under Tab Selection switch off everything except for boundary.
Switch the view from 2D to 3D and remove elev_idw_1m.

In [ ]:
gs.run_command('v.to.3d', input="elev_lid792_randpts", output="elev_lid792_randpts3d", column="value")
gs.run_command('v.delaunay', input="elev_lid792_randpts3d", output="elev_rand_tin")
gs.mapcalc("level90 = 90")

In [ ]:
# end the GRASS session
os.remove(rcfile)